In [1]:
import torchvision.models as models
import torch.onnx as onnx
import torch

/opt/conda/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import PIL
import time
import cv2
import os

In [3]:
from models.experimental import attempt_load
from utils.datasets import letterbox
from utils.general import check_img_size, non_max_suppression_face, apply_classifier, scale_coords, xywh2xyxy, xyxy2xywh, \
    strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

In [4]:
def make_dir(dirName):
    # Create a target directory & all intermediate 
    # directories if they don't exists
    
    if not os.path.exists(dirName):
        os.makedirs(dirName, exist_ok = True)
        print("[INFO] Directory " ,dirName,  " created")
    else:
        print("[INFO] Directory " ,dirName,  " already exists") 

In [5]:
def scale_coords_landmarks(img1_shape, coords, img0_shape, ratio_pad=None):
    # Rescale coords (xyxy) from img1_shape to img0_shape
    
    if ratio_pad is None:  # calculate from img0_shape
        gain = min(img1_shape[0] / img0_shape[0], img1_shape[1] / img0_shape[1])  # gain  = old / new
        pad = (img1_shape[1] - img0_shape[1] * gain) / 2, (img1_shape[0] - img0_shape[0] * gain) / 2  # wh padding
    else:
        gain = ratio_pad[0][0]
        pad = ratio_pad[1]

    coords[:, [0, 2, 4, 6, 8]] -= pad[0]  # x padding
    coords[:, [1, 3, 5, 7, 9]] -= pad[1]  # y padding
    coords[:, :10] /= gain
    coords[:, 0].clamp_(0, img0_shape[1])  # x1
    coords[:, 1].clamp_(0, img0_shape[0])  # y1
    coords[:, 2].clamp_(0, img0_shape[1])  # x2
    coords[:, 3].clamp_(0, img0_shape[0])  # y2
    coords[:, 4].clamp_(0, img0_shape[1])  # x3
    coords[:, 5].clamp_(0, img0_shape[0])  # y3
    coords[:, 6].clamp_(0, img0_shape[1])  # x4
    coords[:, 7].clamp_(0, img0_shape[0])  # y4
    coords[:, 8].clamp_(0, img0_shape[1])  # x5
    coords[:, 9].clamp_(0, img0_shape[0])  # y5
    
    return coords

In [6]:
def show_results(img, xywh, conf, landmarks):
    # Draw detected bounding box with landmarks
    
    h,w,c = img.shape
    
    tl = 1 or round(0.002 * (h + w) / 2) + 1    # line/font thickness
    clors = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(0,255,255)]    # colors for landmarks

    # Convert YOLO coordinates to x1, y1, x2, y2 format
    x1 = int(xywh[0] * w - 0.5 * xywh[2] * w)
    y1 = int(xywh[1] * h - 0.5 * xywh[3] * h)
    x2 = int(xywh[0] * w + 0.5 * xywh[2] * w)
    y2 = int(xywh[1] * h + 0.5 * xywh[3] * h)
    
    # Draw bounding box
    cv2.rectangle(img, (x1,y1), (x2, y2), (0,255,0), thickness=tl, lineType=cv2.LINE_AA)

    # Draw facial landmarks
    for i in range(5):
        point_x = int(landmarks[2 * i] * w)
        point_y = int(landmarks[2 * i + 1] * h)
        cv2.circle(img, (point_x, point_y), tl+1, clors[i], -1)

    # Put confidence threshold
    tf = max(tl - 1, 1)  # font thickness
    label = str(conf)[:5]
    cv2.putText(img, label, (x1, y1 - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
    
    return img

In [7]:
def detect_one(model, image_path, device, imgsz=640, conf_thres=0.5, iou_thres=0.5, augment=None, show=False):
    # Detect face region and facial landmarks on given images
    
    # Load image and create a copy
    orgimg = cv2.imread(image_path)
    img0 = orgimg.copy()
    
    # Preprocess the image     
    stride = int(model.stride.max())    
    if imgsz <= 0:                    
        imgsz = dynamic_resize(img0.shape)
    imgsz = check_img_size(imgsz, s=64)
    img = letterbox(img0, imgsz)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)
    img = img.float()
    img /= 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    if augment is not None:
        pred = model(img, augment=True, aug_setting=augment)[0]
    else:
        pred = model(img, augment=False)[0] 
    
    # Apply non max suppression
    pred = non_max_suppression_face(pred, conf_thres, iou_thres)[0]
    gn = torch.tensor(img0.shape)[[1, 0, 1, 0]].to(device)                          # normalization gain whwh
    gn_lks = torch.tensor(img0.shape)[[1, 0, 1, 0, 1, 0, 1, 0, 1, 0]].to(device)    # normalization gain landmarks
    
    # Scale and store predictions
    boxes = []
    landmarks = []
    conf_scores = []
    h, w, c = img0.shape
    if pred is not None:
        pred[:, :4] = scale_coords(img.shape[2:], pred[:, :4], img0.shape).round()
        pred[:, 5:15] = scale_coords_landmarks(img.shape[2:], pred[:, 5:15], img0.shape).round()
        for j in range(pred.size()[0]):
            xywh = (xyxy2xywh(pred[j, :4].view(1, 4)) / gn).view(-1)
            xywh = xywh.data.cpu().numpy()
            conf = pred[j, 4].cpu().numpy()
            points = (pred[j, 5:15].view(1, 10) / gn_lks).view(-1).tolist()
            class_num = pred[j, 15].cpu().numpy()
            x1 = int(xywh[0] * w - 0.5 * xywh[2] * w)
            y1 = int(xywh[1] * h - 0.5 * xywh[3] * h)
            x2 = int(xywh[0] * w + 0.5 * xywh[2] * w)
            y2 = int(xywh[1] * h + 0.5 * xywh[3] * h)
            
            if show:
                orgimg = show_results(orgimg, xywh, conf, points)
                
            boxes.append([x1, y1, x2-x1, y2-y1])
            landmarks.append(points)
            conf_scores.append(conf)
            
    if show:
        img = PIL.Image.fromarray(cv2.cvtColor(orgimg, cv2.COLOR_BGR2RGB))
        img.show()
        
    return boxes, landmarks, conf_scores

In [8]:
def evaluate(imagePaths, res_path, model, device, img_sizes, con_thr, iou_thr, augment=None):
    # Evaluate model performance on a dataset

    for ind, imagePath in enumerate(imagePaths,1):
        print("Processing image: {}/{}".format(ind, len(imagePaths)))
        
        # Detect
        boxes, pred_landmarks, scores = detect_one(model, imagePath, device, 
                                                                 img_size, con_thr, iou_thr, augment=augment)

        # Open the file to  write the results in WIDER Face submission format
        p1, p2 = imagePath.split('/')[-2:]
        imageName = p2.split('.')[0]
        
        make_dir(os.path.join(res_path, p1))
        
        det_file = open("{}/{}/{}.txt".format(res_path, p1, imageName),"w")
        det_file.write(imageName)
        det_file.write("\n")
        det_file.write(str(len(boxes)))
        det_file.write("\n")
    
        # iterate through the detected bounding boxes and conf scores
        for pred_box, conf_score in zip(boxes, scores):
            det_file.write(str(int(pred_box[0])))
            det_file.write(" ")
            det_file.write(str(int(pred_box[1])))
            det_file.write(" ")
            det_file.write(str(int(pred_box[2])))
            det_file.write(" ")
            det_file.write(str(int(pred_box[3])))
            det_file.write(" ")
            det_file.write(str(conf_score))
            det_file.write("\n")
                
        det_file.close()

In [12]:
exp_id = 502            # experiment ID
iou_thr = 0.5           # IoU threshold
con_thr = 0.02          # confidence threshold
img_size = 640         # image size used at evaluation

set_ = 'val'            # val / test / train
weights = "/workspace/yolov5-face-my/runs/train/exp52/weights/best.pt"    # path to the model weights

# Get paths to the dataset images
wf_dir = '/workspace/yolov5-face/widerface/'
wf_paths = list(paths.list_images(os.path.join(wf_dir, set_, 'images')))

print("WiderFace {} set size: {}". format(set_, len(wf_paths)))

# Create directory for saving evaluation results
wf_res_path = "Object-Detection-Metrics/results/{}/exp_{}/widerface/preds_{}_{}".format(set_, exp_id, 
                                                                                     int(iou_thr * 100),
                                                                                     int(con_thr * 100))

make_dir(wf_res_path)

WiderFace val set size: 3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2  already exists


In [13]:
# Load weights 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = attempt_load(weights, map_location=device)

Fusing layers... 


In [15]:
evaluate(wf_paths, wf_res_path, model, device, img_size, con_thr, iou_thr, augment=None)

Processing image: 1/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/31--Waiter_Waitress  already exists
Processing image: 2/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/31--Waiter_Waitress  already exists
Processing image: 3/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/31--Waiter_Waitress  already exists
Processing image: 4/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/31--Waiter_Waitress  already exists
Processing image: 5/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/31--Waiter_Waitress  already exists
Processing image: 6/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/31--Waiter_Waitress  already exists
Processing image: 7/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/31--Waiter_Waitress  

[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 61/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 62/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 63/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 64/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 65/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 66/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 67/3226

[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 122/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 123/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 124/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 125/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 126/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 127/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 1

[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 183/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 184/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 185/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 186/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 187/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 188/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 1

[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 240/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 241/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 242/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 243/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 244/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 245/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/2--Demonstration  already exists
Processing image: 2

...

[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/16--Award_Ceremony  already exists
Processing image: 2938/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/16--Award_Ceremony  already exists
Processing image: 2939/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/16--Award_Ceremony  already exists
Processing image: 2940/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/16--Award_Ceremony  already exists
Processing image: 2941/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/16--Award_Ceremony  already exists
Processing image: 2942/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/16--Award_Ceremony  already exists
Processing image: 2943/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/16--Award_Ceremony  already exists

[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/50--Celebration_Or_Party  already exists
Processing image: 2993/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/50--Celebration_Or_Party  already exists
Processing image: 2994/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/50--Celebration_Or_Party  already exists
Processing image: 2995/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/50--Celebration_Or_Party  already exists
Processing image: 2996/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/50--Celebration_Or_Party  already exists
Processing image: 2997/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/50--Celebration_Or_Party  already exists
Processing image: 2998/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_

[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3055/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3056/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3057/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3058/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3059/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3060/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3061/3226

Processing image: 3112/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3113/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3114/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3115/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3116/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3117/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3118/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists

[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3172/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3173/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3174/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3175/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3176/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3177/3226
[INFO] Directory  Object-Detection-Metrics/results/val/exp_502/widerface/preds_50_2/35--Basketball  already exists
Processing image: 3178/3226